In [4]:
import numpy as np
import jax.numpy as jnp
!pip install matplotlib


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 50.1 MB/s  0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 59.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [matplotlib] 6/7 [matplotlib]ow]


In [24]:
from langchain_ibm import ChatWatsonx
from langchain.agents import AgentType
import re
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import Tool

In [11]:
llm = ChatOllama(
    model="qwen2.5:7b-instruct",
    temperature=0
)

In [16]:
response = llm.invoke("What is tool calling")

In [17]:
print("response content:", response.content)

response content: Tool calling refers to the process of invoking or using external tools from within another program or script. This can be done in various programming languages and environments where you want to leverage the functionality provided by an external application, library, or utility.

Here are some key aspects of tool calling:

1. **Purpose**: Tool calling is often used when a specific task requires specialized software that isn't natively supported within the main program. For example, using a text processing tool from Python or running a machine learning model in R from a Python script.

2. **Methods**:
   - **Subprocesses**: In many programming languages (like Python), you can use subprocess modules to call external programs and capture their output.
   - **System Calls**: Directly calling system commands using the `system()` function or similar methods.
   - **APIs**: Using APIs provided by tools, which might be more integrated into your language of choice.

3. **Examp

In [21]:
def add_numbers(inputs:str)->dict:
    """
    Adds a list of numbers provided in the input dictionary or extracts numbers from a string
    Parameters
    - inputs (str):
    string, it should contain numbers that can extracted adn summed

    Returns:
    - dict: A dictionary with a single key, "result", conatinsing sum of two numbers.
    """

    numbers = [int(x) for x in inputs.replace(",","").split() if x.isdigit()]

    result = sum(numbers)
    return {"result":result}

In [23]:
add_numbers("Find the sum of 2 and 22222")

{'result': 22224}

In [25]:
add_tool = Tool(
    name="AddTool",
    func = add_numbers,
    description = "Adds a list of numbers and returns the result.")


In [26]:
print("tool object", add_tool)

tool object name='AddTool' description='Adds a list of numbers and returns the result.' func=<function add_numbers at 0x319b18680>


# Lets try a simple add

In [34]:
test_input = "Please add 60 and 20 for me. Add 120 to the result"
print(add_tool.invoke(test_input))

{'result': 200}


In [30]:
add_tool.description

'Adds a list of numbers and returns the result.'

In [31]:
add_tool.name

'AddTool'

In [32]:
add_tool.args

{'tool_input': {'type': 'string'}}